# Expresiones regulares como fuente de datos faltantes
---

### Importo, preparo y cargo df

In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
df_propierati = pd.read_csv('../properati.zip', compression='zip')
#df_prop_uz = pd.read_csv('../properati.csv')

In [3]:
pd.set_option('display.max_colwidth', 1500)
pd.set_option('display.max_rows', 200)

## Preguntas iniciales
---

In [4]:
df_propierati.columns

Index(['Unnamed: 0', 'operation', 'property_type', 'place_name',
       'place_with_parent_names', 'country_name', 'state_name', 'geonames_id',
       'lat-lon', 'lat', 'lon', 'price', 'currency',
       'price_aprox_local_currency', 'price_aprox_usd', 'surface_total_in_m2',
       'surface_covered_in_m2', 'price_usd_per_m2', 'price_per_m2', 'floor',
       'rooms', 'expenses', 'properati_url', 'description', 'title',
       'image_thumbnail'],
      dtype='object')

In [5]:
df_propierati.dtypes

Unnamed: 0                      int64
operation                      object
property_type                  object
place_name                     object
place_with_parent_names        object
country_name                   object
state_name                     object
geonames_id                   float64
lat-lon                        object
lat                           float64
lon                           float64
price                         float64
currency                       object
price_aprox_local_currency    float64
price_aprox_usd               float64
surface_total_in_m2           float64
surface_covered_in_m2         float64
price_usd_per_m2              float64
price_per_m2                  float64
floor                         float64
rooms                         float64
expenses                      float64
properati_url                  object
description                    object
title                          object
image_thumbnail                object
dtype: objec

## Regex
---

Patrones

In [39]:
# Precios inmueble
precios_inmueble_pat = '(usd|u\$d|u\$s|us\$|dolares) ?(\d{2,}(\,|\.)\d{0,3}\,?\.?\d{0,3}\,?\.?\d{0,3}) ?(?!(.*m2|.*mc|.*metros))'
precios_inmueble_reg = re.compile(precios_inmueble_pat, flags = re.IGNORECASE)

# Metros inmueble
metros_inmueble_pat = '(\d{0,4}\,?\.?\d{0,4})? ?(m2|mc|metros|mts)'
metros_inmueble_reg = re.compile(metros_inmueble_pat, flags = re.IGNORECASE)

# Cochera
cochera_pat = '(?<!sin )(?<!no incluye )(?<!no posee )cochera(?! no incluye)(?! no incluida)'
cochera_reg = re.compile(cochera_pat, flags = re.IGNORECASE)

# Piscina/Pileta
piscina_pat = 'piscina|pileta(?! de baño)(?! de cocina)'
piscina_reg = re.compile(piscina_pat, flags = re.IGNORECASE)

# Ambientes
amb_pat = '(\d{1,4})(?= ambientes)(?= amb)'
amb_reg = re.compile(amb_pat, flags = re.IGNORECASE)

## Regex ambientes
---

In [29]:
df_propierati_rooms_null_mask = df_propierati.rooms.isnull()
df_propierati.rooms.isnull().value_counts()

True     73830
False    47390
Name: rooms, dtype: int64

In [30]:
df_propierati.loc[df_propierati_rooms_null_mask, 'rooms'].head(2)

0   NaN
1   NaN
Name: rooms, dtype: float64

In [31]:
df_propierati_titl_notnull_mask = df_propierati.title.notnull()
df_propierati_desc_notnull_mask = df_propierati.description.notnull()

In [32]:
df_propierati_rooms_titl_boolean = df_propierati_rooms_null_mask & df_propierati_titl_notnull_mask
df_propierati_rooms_desc_boolean = df_propierati_rooms_null_mask & df_propierati_desc_notnull_mask

In [33]:
df_propierati_rooms_titl_boolean.value_counts()
df_propierati_rooms_desc_boolean.value_counts()

True     73828
False    47392
dtype: int64

In [49]:
df_propierati['rooms_regex_desc'] = df_propierati.description.apply(lambda x: x if x is None or np.NaN else amb_reg.search(x))
df_propierati['rooms_regex_desc'] = df_propierati.rooms_regex_desc.apply(lambda x: x if x is None or np.NaN else x.group(1))

In [45]:
df_propierati['rooms_regex_titl'] = df_propierati.title.apply(lambda x: x if x is np.NaN else amb_reg.search(x))
df_propierati['rooms_regex_titl'] = df_propierati.rooms_regex_titl.apply(lambda x: x if x is None else x.group(1))
df_propierati.title.apply(lambda x: x if x is np.NaN else amb_reg.search(x))

0                                                None
1                                                None
2                                                None
3                                                None
4                                                None
                             ...                     
121215    <re.Match object; span=(31, 32), match='3'>
121216                                           None
121217                                           None
121218                                           None
121219                                           None
Name: title, Length: 121220, dtype: object

In [42]:
df_propierati.rooms_regex_titl.value_counts()

2       4372
3       4137
4       2298
5        697
6        194
7         66
8         23
1         15
9          7
11         3
10         1
14         1
12         1
4000       1
Name: rooms_regex_titl, dtype: int64

In [43]:
df_propierati.rooms_regex_desc.value_counts()

AVISO LEGAL: Las descripciones arquitectónicas y funcionales, valores de expensas, impuestos y servicios, fotos y medidas de este inmueble son aproximados. Los datos fueron proporcionados por el propietario y pueden no estar actualizados a la hora de la visualización de este aviso por lo cual pueden arrojar inexactitudes y discordancias con las que surgen de los las facturas, títulos y planos legales del inmueble. Los precios indicados podrán modificarse sin previo aviso. La fechas de inicio de obra o posesión son estimadas, podrán ser reprogramadas por la Dirección de obra y dependerán a su vez de un proceso de aprobaciones municipales u otros organismos intervinientes. El interesado deberá realizar las verificaciones respectivas previamente a la realización de cualquier operación, requiriendo por sí o sus profesionales las copias necesarias de la documentación que corresponda. Venta supeditada al cumplimiento por parte del propietario de los requisitos de la resolución general Nº 237

In [ ]:
df_propierati.notnull